## Experiments on Patches of Confocal tyx-Image Series

**Author**: Prisca Dotti

**Last Edit**: 07.06.2024

This notebook is used to run some experiments using the U-Net model on patches obtained from either real or fake confocal imaging data.

The dataset used for training could be one of the following:
- dataset of patches extracted in a meaningful way from confocal imaging recordings
- dataset of recordings from which patches are extracted when processing them in the U-Net for memory management reasons and which are recombined as whole movies after inference
- dataset of simulated confocal imaging patches --> this is similar to what I've been doing so far
- a combination of real and fake patches of confocal imaging data

In [1]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [2]:
import logging
import os
import torch
import random
import numpy as np

from torch import nn, optim

# from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter

import wandb
from config import TrainingConfig, config
from data.datasets import PatchSparkDataset
from models.UNet import unet
from utils.training_inference_tools import (
    MyTrainingManager,
    sampler,
    test_function,
    training_step,
    weights_init,
    TransformedSparkDataset,
    random_flip,
    random_flip_noise,
)
from utils.training_script_utils import (
    get_sample_ids,
    init_criterion,
    init_dataset,
    init_model,
)

logger = logging.getLogger(__name__)

c:\Users\prisc\anaconda3\envs\sparks\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
logger.setLevel(logging.DEBUG)

In [23]:
##################### Get training-specific parameters #####################

# Initialize training-specific parameters
# (get the configuration file path from ArgParse)
config_filename = os.path.join("config_files", "config_patches.ini")
# config_filename = os.path.join("config_files", "config_final_model.ini")

params = TrainingConfig(training_config_file=config_filename)

# Print parameters to console if needed
params.print_params()

######################### Initialize random seeds ##########################

# We used these random seeds to ensure reproducibility of the results

torch.manual_seed(0)  # <--------------------------------------------------!
random.seed(0)  # <--------------------------------------------------------!
np.random.seed(0)  # <-----------------------------------------------------!

[12:48:37] [  INFO  ] [   config   ] <315 > -- Loading C:\Users\prisc\Code\sparks_project\config_files\config_patches.ini
[12:48:37] [  INFO  ] [   config   ] <567 > --     training_config_file: C:\Users\prisc\Code\sparks_project\config_files\config_patches.ini
[12:48:37] [  INFO  ] [   config   ] <567 > --              dataset_dir: C:\Users\prisc\Code\sparks_project\data\sparks_dataset
[12:48:37] [  INFO  ] [   config   ] <567 > --                        c: <configparser.ConfigParser object at 0x000001FEF0C8D820>
[12:48:37] [  INFO  ] [   config   ] <567 > --                 run_name: patches
[12:48:37] [  INFO  ] [   config   ] <567 > --            load_run_name: 
[12:48:37] [  INFO  ] [   config   ] <567 > --               load_epoch: 0
[12:48:37] [  INFO  ] [   config   ] <567 > --             train_epochs: 100000
[12:48:37] [  INFO  ] [   config   ] <567 > --                criterion: lovasz_softmax
[12:48:37] [  INFO  ] [   config   ] <567 > --                 lr_start: 0.0001
[1

In [5]:
# params.set_device("cpu")
params.display_device_info()

[12:38:10] [  INFO  ] [   config   ] <562 > -- Using cuda


In [6]:
############################ Configure datasets ############################

# Select samples for training and testing based on dataset size
train_sample_ids = get_sample_ids(
    train_data=True,
    dataset_size=params.dataset_size,
)
test_sample_ids = get_sample_ids(
    train_data=False,
    dataset_size=params.dataset_size,
)

# Initialize training dataset: here it only samples random patches
dataset = PatchSparkDataset(
    params=params,
    base_path=params.dataset_dir,
    sample_ids=train_sample_ids,
    load_instances=False,
    inference=None,
)
# Apply transforms based on noise_data_augmentation setting
# (transforms are applied when getting a sample from the dataset)
transforms = random_flip_noise if params.noise_data_augmentation else random_flip
dataset = TransformedSparkDataset(dataset, transforms)

logger.info(f"Samples in dataset (patches sampled in an iteration): {len(dataset)}")

# Initialize testing datasets
testing_dataset = init_dataset(
    params=params,
    sample_ids=test_sample_ids,
    apply_data_augmentation=False,
    load_instances=True,
)

[12:38:11] [  INFO  ] [  __main__  ] < 26 > -- Samples in dataset (patches sampled in an iteration): 10000
[12:38:11] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
[12:38:11] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
[12:38:17] [  INFO  ] [utils.training_script_utils] <149 > -- Samples in dataset: 1


In [7]:
# import napari

# train_sample_id = 0
# test_sample_id = 0

# train_sample = dataset[train_sample_id]
# test_sample = testing_dataset[test_sample_id]

# train_data = train_sample["data"].numpy()
# train_labels = train_sample["labels"].numpy()

# test_data = test_sample["data"].numpy()
# test_labels = test_sample["labels"].numpy()

# viewer = napari.Viewer()
# viewer.add_image(train_data)
# viewer.add_labels(train_labels)
# viewer.add_image(test_data)
# viewer.add_labels(test_labels)

In [8]:
# Initialize data loaders
dataset_loader = DataLoader(
    dataset,
    batch_size=params.batch_size,
    num_workers=params.num_workers,
    pin_memory=params.pin_memory,
)

In [9]:
############################## Configure UNet ##############################

# Initialize the UNet model
network = init_model(params=params)

# Move the model to the GPU if available
if params.device.type != "cpu":
    network = nn.DataParallel(network).to(params.device, non_blocking=True)
    # cudnn.benchmark = True

# Watch the model with wandb for logging if enabled
if params.wandb_log:
    wandb.watch(network)

# Initialize UNet weights if required
if params.initialize_weights:
    logger.info("Initializing UNet weights...")
    network.apply(weights_init)

# The following line is commented as it does not work on Windows
# torch.compile(network, mode="default", backend="inductor")

In [20]:
########################### Initialize training ############################

# Initialize the optimizer based on the specified type
if params.optimizer == "adam":
    optimizer = optim.Adam(network.parameters(), lr=params.lr_start)
elif params.optimizer == "adadelta":
    optimizer = optim.Adadelta(network.parameters(), lr=params.lr_start)
elif params.optimizer == "sgd":
    optimizer = optim.SGD(network.parameters(), lr=params.lr_start)
else:
    logger.error(f"{params.optimizer} is not a valid optimizer.")
    exit()

# Initialize the learning rate scheduler if specified
if params.scheduler == "step":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=params.scheduler_step_size,
        gamma=params.scheduler_gamma,
    )
else:
    scheduler = None

# Define the output directory path
output_path = os.path.join(config.output_dir, params.run_name)
logger.info(f"Output directory: {os.path.realpath(output_path)}")

# Initialize the summary writer for TensorBoard logging
summary_writer = SummaryWriter(os.path.join(output_path, "summary"), purge_step=0)

# Check if a pre-trained model should be loaded
if params.load_run_name != "":
    load_path = os.path.join(config.output_dir, params.load_run_name)
    logger.info(f"Model loaded from directory: {os.path.realpath(load_path)}")
else:
    load_path = None

# Initialize the loss function
criterion = init_criterion(params=params, dataset=dataset)

# Create a directory to save predicted class movies
preds_output_dir = os.path.join(output_path, "predictions")
os.makedirs(preds_output_dir, exist_ok=True)

# Create a dictionary of managed objects
managed_objects = {"network": network, "optimizer": optimizer}
if scheduler is not None:
    managed_objects["scheduler"] = scheduler

# Create a training manager with the specified training and testing functions
trainer = MyTrainingManager(
    # Training parameters
    training_step=lambda _: training_step(
        dataset_loader=dataset_loader,
        params=params,
        sampler=sampler,
        network=network,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        # scaler=GradScaler(),
    ),
    save_every=params.c.getint("training", "save_every", fallback=5000),
    load_path=load_path,
    save_path=output_path,
    managed_objects=unet.managed_objects(managed_objects),
    # Testing parameters
    test_function=lambda _: test_function(
        network=network,
        device=params.device,
        criterion=criterion,
        params=params,
        testing_dataset=testing_dataset,
        training_name=params.run_name,
        output_dir=preds_output_dir,
        training_mode=True,
    ),
    test_every=params.c.getint("training", "test_every", fallback=1000),
    plot_every=params.c.getint("training", "test_every", fallback=1000),
    summary_writer=summary_writer,
)

# Load the model if a specific epoch is provided
if params.load_epoch != 0:
    trainer.load(params.load_epoch)

[12:46:15] [  INFO  ] [  __main__  ] < 26 > -- Output directory: C:\Users\prisc\Code\sparks_project\models\saved_models\patches


In [ ]:
# Validate the network before training if resuming from a checkpoint
# if params.load_epoch > 0:
#     logger.info("Validate network before training")
#     trainer.run_validation(wandb_log=params.wandb_log)

In [28]:
# Set the network in training mode
network.train()

# Train the model for the specified number of epochs
logger.info("Starting training")
trainer.train(
    params.train_epochs,
    print_every=params.c.getint("training", "print_every", fallback=100),
    wandb_log=params.wandb_log,
)

[13:11:18] [  INFO  ] [  __main__  ] < 5  > -- Starting training
[13:12:08] [  INFO  ] [utils.training_inference_tools] <1489> -- Iteration 0...
[13:12:08] [  INFO  ] [utils.training_inference_tools] <1490> -- 	Training loss: 0.7575
[13:12:08] [  INFO  ] [utils.training_inference_tools] <1491> -- 	Time elapsed: 2.06s


KeyboardInterrupt: 

In [ ]:
logger.info("Starting final validation")
# Run the final validation/testing procedure
# trainer.run_validation(wandb_log=params.wandb_log)

In [ ]:
# Close the summary writer
summary_writer.close()

# Close the wandb run
if params.wandb_log:
    wandb.finish()